In [25]:
import json
import requests
import numpy as np
import pandas as pd

import requests
from requests.auth import HTTPBasicAuth

credentials = json.loads(open('credentials.json').read())

authentication = HTTPBasicAuth(credentials['username'], credentials['password'])


In [27]:
data = requests.get('https://api.github.com/users/' + credentials['username'],auth = authentication)

data = data.json()

data_l = data

print(data)


{'login': 'Navoxya', 'id': 68534001, 'node_id': 'MDQ6VXNlcjY4NTM0MDAx', 'avatar_url': 'https://avatars.githubusercontent.com/u/68534001?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/Navoxya', 'html_url': 'https://github.com/Navoxya', 'followers_url': 'https://api.github.com/users/Navoxya/followers', 'following_url': 'https://api.github.com/users/Navoxya/following{/other_user}', 'gists_url': 'https://api.github.com/users/Navoxya/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/Navoxya/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/Navoxya/subscriptions', 'organizations_url': 'https://api.github.com/users/Navoxya/orgs', 'repos_url': 'https://api.github.com/users/Navoxya/repos', 'events_url': 'https://api.github.com/users/Navoxya/events{/privacy}', 'received_events_url': 'https://api.github.com/users/Navoxya/received_events', 'type': 'User', 'site_admin': False, 'name': 'Navodya Dinith', 'company': None, 'blog': '', 'location': 'S

In [28]:
print("Information about user {}:\n".format(credentials['username']))
print("Name: {}".format(data['name']))
print("UserName: {}".format(data['login']))
print("Email: {}".format(data['email']))
print("Location: {}".format(data['location']))
print("Public repos: {}".format(data['public_repos']))
print("Public gists: {}".format(data['public_gists']))
print("About: {}".format(data['bio']))

Information about user Navoxya:

Name: Navodya Dinith
UserName: Navoxya
Email: None
Location: Sri Lanka
Public repos: 23
Public gists: 0
About: None


In [29]:
url = data['repos_url']
page_no = 1
repos_data = []
while (True):
    response = requests.get(url,auth = authentication)
    response = response.json()
    repos_data = repos_data + response
    repos_fetched = len(response)
    print("Total repositories fetched: {}".format(repos_fetched))
    if (repos_fetched == 30):
        page_no = page_no + 1
        url = data['repos_url'].encode("UTF-8") + '?page=' + str(page_no)
    else:
        break

Total repositories fetched: 23


In [30]:
repos_information = []
for i, repo in enumerate(repos_data):
    data = []
    data.append(repo['id'])
    data.append(repo['name'])
    data.append(repo['description'])
    data.append(repo['created_at'])
    data.append(repo['updated_at'])
    data.append(repo['owner']['login'])
    data.append(repo['license']['name'] if repo['license'] != None else None)
    data.append(repo['has_wiki'])
    data.append(repo['forks_count'])
    data.append(repo['open_issues_count'])
    data.append(repo['stargazers_count'])
    data.append(repo['watchers_count'])
    data.append(repo['url'])
    data.append(repo['commits_url'].split("{")[0])
    data.append(repo['url'] + '/languages')
    repos_information.append(data)
    
repos_df = pd.DataFrame(repos_information, columns = ['Id', 'Name', 'Description', 'Created on', 'Updated on', 
                                                      'Owner', 'License', 'Includes wiki', 'Forks count', 
                                                      'Issues count', 'Stars count', 'Watchers count',
                                                      'Repo URL', 'Commits URL', 'Languages URL'])
# repos_df.to_csv('repos_info.csv', index = False)


In [31]:
for i in range(repos_df.shape[0]):
    response = requests.get(repos_df.loc[i, 'Languages URL'], auth = authentication)
    response = response.json()
    print(i, response)
    if response != {}:
        languages = []
        values = []
        for key, value in response.items():
            languages.append(key)
            values.append(value)
        languages = ', '.join(languages)
#         values = ', '.join(values)
        repos_df.loc[i, 'Languages'] = languages
#         repos_df.loc[i, 'Languages Values'] = values
    else:
        repos_df.loc[i, 'Languages'] = ""
#         repos_df.loc[i, 'Languages Values'] = ""
        
repos_df.to_csv('repos_info.csv', index = False)

0 {'JavaScript': 24131, 'HTML': 1078, 'CSS': 564}
1 {'JavaScript': 19841, 'HTML': 1084, 'CSS': 564, 'Shell': 103}
2 {'JavaScript': 12760}
3 {'JavaScript': 24840}
4 {'JavaScript': 15447}
5 {'JavaScript': 46400, 'CSS': 8794, 'HTML': 682}
6 {'JavaScript': 53819, 'SCSS': 2832, 'HTML': 1846, 'CSS': 475}
7 {'JavaScript': 80045, 'CSS': 22003, 'HTML': 1820}
8 {'SCSS': 83932, 'JavaScript': 47192}
9 {'HTML': 677}
10 {}
11 {'Java': 5715}
12 {'HTML': 67118, 'CSS': 15701, 'JavaScript': 3955}
13 {'Java': 144827}
14 {'Java': 40399}
15 {'Java': 44466}
16 {'HTML': 230607, 'CSS': 148220, 'JavaScript': 100991, 'Java': 63267, 'SCSS': 47205, 'Less': 5937, 'Ruby': 161}
17 {'Java': 3866}
18 {'JavaScript': 13890}
19 {'JavaScript': 63438, 'CSS': 14708, 'HTML': 1820}
20 {'Dart': 2556651, 'JavaScript': 4005, 'HTML': 1130, 'Shell': 571, 'Swift': 404, 'Kotlin': 127, 'Objective-C': 38}
21 {'JavaScript': 76959}
22 {'Java': 2816}


In [32]:
langArr = []

for i in range(repos_df.shape[0]):
    response = requests.get(repos_df.loc[i, 'Languages URL'], auth = authentication)
    response = response.json()
    print(i, response)
    if response != {}:
        arr = []
        for key, value in response.items():
            b = {key : value}
            arr.append(b)
        
        langArr.append(arr)
        
    else:
        langArr.append([])
        
langArr

0 {'JavaScript': 24131, 'HTML': 1078, 'CSS': 564}
1 {'JavaScript': 19841, 'HTML': 1084, 'CSS': 564, 'Shell': 103}
2 {'JavaScript': 12760}
3 {'JavaScript': 24840}
4 {'JavaScript': 15447}
5 {'JavaScript': 46400, 'CSS': 8794, 'HTML': 682}
6 {'JavaScript': 53819, 'SCSS': 2832, 'HTML': 1846, 'CSS': 475}
7 {'JavaScript': 80045, 'CSS': 22003, 'HTML': 1820}
8 {'SCSS': 83932, 'JavaScript': 47192}
9 {'HTML': 677}
10 {}
11 {'Java': 5715}
12 {'HTML': 67118, 'CSS': 15701, 'JavaScript': 3955}
13 {'Java': 144827}
14 {'Java': 40399}
15 {'Java': 44466}
16 {'HTML': 230607, 'CSS': 148220, 'JavaScript': 100991, 'Java': 63267, 'SCSS': 47205, 'Less': 5937, 'Ruby': 161}
17 {'Java': 3866}
18 {'JavaScript': 13890}
19 {'JavaScript': 63438, 'CSS': 14708, 'HTML': 1820}
20 {'Dart': 2556651, 'JavaScript': 4005, 'HTML': 1130, 'Shell': 571, 'Swift': 404, 'Kotlin': 127, 'Objective-C': 38}
21 {'JavaScript': 76959}
22 {'Java': 2816}


[[{'JavaScript': 24131}, {'HTML': 1078}, {'CSS': 564}],
 [{'JavaScript': 19841}, {'HTML': 1084}, {'CSS': 564}, {'Shell': 103}],
 [{'JavaScript': 12760}],
 [{'JavaScript': 24840}],
 [{'JavaScript': 15447}],
 [{'JavaScript': 46400}, {'CSS': 8794}, {'HTML': 682}],
 [{'JavaScript': 53819}, {'SCSS': 2832}, {'HTML': 1846}, {'CSS': 475}],
 [{'JavaScript': 80045}, {'CSS': 22003}, {'HTML': 1820}],
 [{'SCSS': 83932}, {'JavaScript': 47192}],
 [{'HTML': 677}],
 [],
 [{'Java': 5715}],
 [{'HTML': 67118}, {'CSS': 15701}, {'JavaScript': 3955}],
 [{'Java': 144827}],
 [{'Java': 40399}],
 [{'Java': 44466}],
 [{'HTML': 230607},
  {'CSS': 148220},
  {'JavaScript': 100991},
  {'Java': 63267},
  {'SCSS': 47205},
  {'Less': 5937},
  {'Ruby': 161}],
 [{'Java': 3866}],
 [{'JavaScript': 13890}],
 [{'JavaScript': 63438}, {'CSS': 14708}, {'HTML': 1820}],
 [{'Dart': 2556651},
  {'JavaScript': 4005},
  {'HTML': 1130},
  {'Shell': 571},
  {'Swift': 404},
  {'Kotlin': 127},
  {'Objective-C': 38}],
 [{'JavaScript': 769

In [35]:
usage_information = []

# repos_data[22]['name']
# data['login']

for i, repo in enumerate(langArr):

    for j, usage in enumerate(repo):

        for key, value in usage.items():
            dataU = []
            dataU.append(repos_data[i]['id'])
            dataU.append(data_l['login'])
            dataU.append(repos_data[i]['name'])
            dataU.append(key)
            dataU.append(value)
            usage_information.append(dataU)
            
usage_df = pd.DataFrame(usage_information, columns = ['Id', 'User','Name','Language','value'])
usage_df.to_csv('tech_usageV2.csv', index = False)
usage_df.to_json('tech_usageV2.json', orient='columns')
usage_df

,Id,User,Name,Language,value
0,376543369,Navoxya,AF-final,JavaScript,24131
1,376543369,Navoxya,AF-final,HTML,1078
2,376543369,Navoxya,AF-final,CSS,564
3,376376615,Navoxya,AF-paper,JavaScript,19841
4,376376615,Navoxya,AF-paper,HTML,1084
5,376376615,Navoxya,AF-paper,CSS,564
6,376376615,Navoxya,AF-paper,Shell,103
7,382661686,Navoxya,api-ICAF,JavaScript,12760
8,405410677,Navoxya,api-SchoolMS-1,JavaScript,24840
9,397928931,Navoxya,api-shopping-store,JavaScript,15447
